In [9]:
from langchain_community.document_loaders.pdf import PyPDFLoader

path = '../documents/guia_alimentar_populacao_brasileira.pdf'
loader = PyPDFLoader(path)

pdf_pages = loader.load()

### Faz splitting do PDF:

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=70,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = splitter.split_documents(pdf_pages)

### Criando a Vector Store persistida em arquivo

In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

chroma_location = "../documents/chroma_vectorstore/"

vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=chroma_location
)

# Printa quantidade de documentos
print(vector_store._collection.count())


1839


### Importando a Vector Store do Disco

In [19]:
vector_store = Chroma(
    persist_directory=chroma_location,
    embedding_function=embedding_model
)

question = "O que fala sobre arroz?"

docs = vector_store.similarity_search(question, k=5) # k=5 para retornar os 5 documentos mais similares

for doc in docs:
    print(doc.page_content)
    print(f'======== {doc.metadata} \n')


arroz
o arroz é o principal representante do grupo dos cereais no Brasil. Como mencionado, o consumo mais frequente é na mistura com o feijão. mas, sendo um alimento extremamente versátil, é também consumido em preparações com legumes, verduras, ovos e carnes, como em vários tipos de risoto, arroz à grega, arroz de cuxá, arroz carreteiro, galinhada e maria izabel. o  
arroz também é ingrediente de doces tradicionais brasileiros como o arroz-doce ou arroz de leite.
======== {'page': 68, 'source': '../documents/guia_alimentar_populacao_brasileira.pdf'} 

arroz
o arroz é o principal representante do grupo dos cereais no Brasil. Como mencionado, o consumo mais frequente é na mistura com o feijão. mas, sendo um alimento extremamente versátil, é também consumido em preparações com legumes, verduras, ovos e carnes, como em vários tipos de risoto, arroz à grega, arroz de cuxá, arroz carreteiro, galinhada e maria izabel. o  
arroz também é ingrediente de doces tradicionais brasileiros como o ar